In [64]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# using BeautifulSoup to get the page content

In [90]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').content
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, 'html.parser')
#soup#.prettify()

# taking the table out of the html content

In [92]:
all_tables=soup.find_all("table")
right_table=soup.find('table', class_='wikitable sortable')
A,B,C=[],[],[]
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True).replace('\n',''))
        B.append(cells[1].find(text=True).replace('\n',''))
        C.append(cells[2].find(text=True).replace('\n',''))

# creating a dataframe from the page extracted

In [126]:
df=pd.DataFrame()
df['PostalCode']=A
df['Borough']=B
df['Neighborhood']=C
df = df[df['Borough'] != 'Not assigned']
len(df[df['Neighborhood'] == 'Not assigned']) == 0 # every neighborhood is assigned
df.shape

(103, 3)

In [103]:
df

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


# adding latitude and longitude to the dataframe

In [130]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
df['Latitude'] = None
df['Longitude'] = None
# loop until you get the coordinates
for i,postal_code in enumerate(df['PostalCode']):
    print(i)
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    df['Latitude'].iloc[i] = latitude
    df['Longitude'].iloc[i] = longitude

0


KeyboardInterrupt: 

In [133]:

df['Latitude'] = pd.read_csv('./data/Geospatial_Coordinates.csv')['Latitude']
df['Longitude'] = pd.read_csv('./data/Geospatial_Coordinates.csv')['L']

0      43.806686
1      43.784535
2      43.763573
3      43.770992
4      43.773136
         ...    
98     43.706876
99     43.696319
100    43.688905
101    43.739416
102    43.706748
Name: Latitude, Length: 103, dtype: float64